In [12]:
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
client = openai.OpenAI()

In [13]:
vector_store = client.vector_stores.create(name='Apostila Asimov')

In [14]:
files = ["Explorando a API da OpenAI.pdf",
         "Explorando o Universo das IAs com Hugging Face.pdf"]
file_stream = [open(f, 'rb') for f in files]
file_batch = client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream
)

In [15]:
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [16]:
assistant = client.beta.assistants.create(
    name='Tutor Asimov',
    instructions="Você é um tutor de uma escola de programação. Você é ótimo para responder perguntas teóricas sobre a api da OpenAI e sobre a utilização da biblioteca do Hugging Face \
        com Python. Você utiliza as apostilas dos cursos para basear suas respostas. Caso você não encontre as respostas nas apostilas informadas, você fala que não sabe responder.",
    tools=[{'type': 'file_search'}],
    tool_resources={'file_search': {'vector_store_ids': [vector_store.id]}},
    model='gpt-4o-mini'
)

In [17]:
thread = client.beta.threads.create()

In [37]:
mensagem_texto = 'Segundo o documento fornecido, como utilizar assistants com python?'
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content=mensagem_texto
)

In [38]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuário é Gabriel e ele é um usuário Premium.'
)

In [39]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
print(run.status)

completed


In [40]:
run.status

'completed'

In [41]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print('Erro', run.status)

SyncCursorPage[Message](data=[Message(id='msg_hbTdCl1V2gpLzPYKXGl93qdi', assistant_id='asst_dbjoZDBFG8387TmE3M6iuANn', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=2465, file_citation=FileCitation(file_id='file-Vy6vEBGPndopncE9y9ojff'), start_index=2453, text='【8:4†source】', type='file_citation')], value='Para utilizar assistentes com Python, você pode seguir os passos descritos no documento, que incluem a criação, configuração e interação com assistentes inteligentes através da API da OpenAI. Aqui está um resumo do que fazer:\n\n### 1. Inicializando o Cliente da OpenAI\nPrimeiro, inicialize o cliente da OpenAI utilizando a biblioteca `openai` e carregando suas credenciais através do `dotenv`. O código para isso é:\n\n```python\nimport openai\nfrom dotenv import load_dotenv, find_dotenv\n\n_ = load_dotenv(find_dotenv())\nclient = openai.Client()\n```\n\n### 2. Criando um Assistente\nEm seguida, você pode criar um a

In [42]:
print(mensagens.data[0].content[0].text.value)

# 【4:2†source】: isso indica que o agente utilizou informações da vector_store para formular a frase.

Para utilizar assistentes com Python, você pode seguir os passos descritos no documento, que incluem a criação, configuração e interação com assistentes inteligentes através da API da OpenAI. Aqui está um resumo do que fazer:

### 1. Inicializando o Cliente da OpenAI
Primeiro, inicialize o cliente da OpenAI utilizando a biblioteca `openai` e carregando suas credenciais através do `dotenv`. O código para isso é:

```python
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
client = openai.Client()
```

### 2. Criando um Assistente
Em seguida, você pode criar um assistente com uma instrução específica. Por exemplo, um assistente de matemática pode ser criado da seguinte forma:

```python
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo-preview",
)
```

### 3. Criando uma Thr

In [43]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [44]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            if tool_call.type == 'file_search':
                print(tool_call)
            else:
                print('-----')
                print(tool_call.code_interpreter.input)
                print('-----')
                print('Result')
                # Certifique-se de que outputs não esteja vazio antes de acessar
                if tool_call.code_interpreter.outputs:
                    print(tool_call.code_interpreter.outputs[0].logs)
                else:
                    print("No outputs available.")
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)


=== Step: tool_calls
FileSearchToolCall(id='call_KIw9ghXmVtVWQUFTANsGcd8N', file_search=FileSearch(ranking_options=FileSearchRankingOptions(ranker='default_2024_08_21', score_threshold=0.0), results=[FileSearchResult(file_id='file-Vy6vEBGPndopncE9y9ojff', file_name='Explorando a API da OpenAI.pdf', score=0.9039121805384137, content=None, attributes={}), FileSearchResult(file_id='file-Vy6vEBGPndopncE9y9ojff', file_name='Explorando a API da OpenAI.pdf', score=0.8997835448118653, content=None, attributes={}), FileSearchResult(file_id='file-Vy6vEBGPndopncE9y9ojff', file_name='Explorando a API da OpenAI.pdf', score=0.8616811316037848, content=None, attributes={}), FileSearchResult(file_id='file-Vy6vEBGPndopncE9y9ojff', file_name='Explorando a API da OpenAI.pdf', score=0.7857413083505586, content=None, attributes={}), FileSearchResult(file_id='file-Vy6vEBGPndopncE9y9ojff', file_name='Explorando a API da OpenAI.pdf', score=0.776539575243294, content=None, attributes={}), FileSearchResult(fil

In [45]:
mensagens = client.beta.threads.messages.list(
    thread_id=thread.id
)
mensagem = list(mensagens)[0].content[0].text
anotacoes = mensagem.annotations
citacoes = []
for index, anotacao in enumerate(anotacoes):
    mensagem.value = mensagem.value.replace(anotacao.text, f'[{index}]')
    if file_cit := getattr(anotacao, 'file_citation', None):
        file = client.files.retrieve(file_cit.file_id)
        citacoes.append("[{}] {}".format(index, file.filename))
citacoes = "\n".join(citacoes)
mensagem.value = f'{mensagem.value}\n\n{citacoes}'

In [46]:
print(mensagem.value)

Para utilizar assistentes com Python, você pode seguir os passos descritos no documento, que incluem a criação, configuração e interação com assistentes inteligentes através da API da OpenAI. Aqui está um resumo do que fazer:

### 1. Inicializando o Cliente da OpenAI
Primeiro, inicialize o cliente da OpenAI utilizando a biblioteca `openai` e carregando suas credenciais através do `dotenv`. O código para isso é:

```python
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
client = openai.Client()
```

### 2. Criando um Assistente
Em seguida, você pode criar um assistente com uma instrução específica. Por exemplo, um assistente de matemática pode ser criado da seguinte forma:

```python
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo-preview",
)
```

### 3. Criando uma Thr